In [1]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torch
import numpy as np
import yaml
import os
import randomname
import random
import utils
from torchvision import transforms
from torch.utils.data import Dataset
from torch import Tensor
from PIL import Image
from siamese import SiameseNetwork
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from contrastive_loss import ContrastiveLoss
import torch.nn.functional as F

2023-08-04 16:51:12.289075: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-04 16:51:12.335783: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 16:51:13.064943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
y2 = torch.ones(32, 8) * 4990
y3 = torch.ones(32, 8) * 5000
loss = ContrastiveLoss()
loss(y2, y3, 0)

tensor(0.)

In [28]:
if __name__ == "__main__":
    torch.cuda.empty_cache()

    # Open config
    with open('../config.yaml', 'r') as file:
        args = yaml.safe_load(file)

    # Assign a unique folder as training output
    training_id = f'{randomname.get_name()}-{str(random.randint(1,9))}'

    final_path = os.path.join(
        '../', args['output_path'], training_id)

    while os.path.isdir(final_path):
        training_id = f'{randomname.get_name()}-{str(random.randint(1,9))}'
        final_path = os.path.join(
            '../', args['output_path'], training_id)

    os.makedirs(os.path.join('../', final_path), exist_ok=True)

    # Write config to output folder
    with open(os.path.join('../', final_path, 'config.yaml'), 'w') as file:
        yaml.dump(args, file)

    # Set device to CUDA if a CUDA device is available, else CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Get image paths and labels from dataset
    dataset_path = os.path.join('../', args['dataset_dir'], args['dataset_name'])
    file_paths, labels = utils.getDataset(dataset_path)

    # Split image paths and labels using Stratified
    files_train, labels_train, files_test, labels_test = utils.stratifiedSortedSplit(
        file_paths, labels, args['train_size'], args['test_size'], args['random_seed'])

In [3]:
class SiameseDataset(Dataset):
    def __init__(self, image_paths: np.array, image_classes: Tensor, 
                shuffle_pairs: bool, augment: bool, final_shape: tuple, grayscale: bool):

        self.image_paths = image_paths
        self.image_classes = image_classes

        num_channels = 1 if grayscale else 3
        self.shuffle_pairs = shuffle_pairs

        if augment:
            # If images are to be augmented, add extra operations for it (first two).
            self.transform = transforms.Compose([
                # transforms.RandomAffine(degrees=20, translate=(
                #     0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
                # transforms.RandomHorizontalFlip(p=0.5),
                transforms.TrivialAugmentWide(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                        0.229, 0.224, 0.225]),
                transforms.Resize(final_shape, antialias=None)
            ])
        else:
            # If no augmentation is needed then apply only the normalization and resizing operations.
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                        0.229, 0.224, 0.225]),
                transforms.Resize(final_shape, antialias=None)
            ])
            
        self.images = torch.empty(len(self.image_paths), num_channels, 320, 280)

        for idx, image_path in enumerate(self.image_paths):
            image = Image.open(image_path).convert("RGB")
            image = self.transform(image).float()
            self.images[idx] = image  
            

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        return self.images[index], self.image_classes[index]

In [4]:
train_dataset = SiameseDataset(files_train, labels_train, **args['train_dataset'])
train_dataloader = DataLoader(train_dataset, **args['train_dataloader'])
val_dataset = SiameseDataset(files_test, labels_test, **args['test_dataset'])
val_dataloader = DataLoader(val_dataset, **args['val_dataloader'])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [6]:
model = SiameseNetwork()

model_dict = torch.load('../reports/milky-arch-3/best.pt')
model.load_state_dict(model_dict['model_state_dict'])
model.to(device)

/home/krisandre7/mambaforge/envs/iris/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/krisandre7/mambaforge/envs/iris/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SiameseNetwork(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [20]:
def get_image_embeds(dataloader: DataLoader):
    start = 0
    end = 0
    
    i = 0
    if dataloader.drop_last:
        size = ((len(dataloader.dataset) - 1) // dataloader.batch_size) * dataloader.batch_size
    else:
        size = len(dataloader.dataset)
    image_embeds = torch.empty(size, 8)
    image_labels = torch.empty(size)
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            image_embed = model.backbone(images)
            image_embed = model.cls_head(image_embed)
            
            end = len(labels)
            end += start 
            
            image_labels[start:end] = labels
            image_embeds[start:end] = image_embed
            start = end
            
    return image_embeds, image_labels

X_train, y_train = get_image_embeds(train_dataloader)
X_test, y_test = get_image_embeds(val_dataloader)

In [26]:
clf = svm.SVC()
clf.fit(X_train.cpu(), y_train.cpu())
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.1834862385321101

In [35]:
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.2018348623853211


In [56]:
lgboost = lgb.LGBMClassifier()
lgboost.fit(X_train.cpu().numpy(), y_train.cpu().numpy().astype(int))
y_pred = lgboost.predict(X_test.cpu().numpy())

y_pred
print(accuracy_score(y_test, y_pred))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000076 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 192, number of used features: 8
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM] [Info] Start training from score -4.564348
[LightGBM]